# Lab: Configure environment and Virtual Machine

In this lab, you will:
- download and install Docker
- build and run Docker containers
- use Ansible to provision Docker containers on your machine
- use Ansible to privision a cluster of machines on Amazon's Elastic Compute (EC2)

First, the following instructions on the Docker documentation site will guide you through installing and initial setup of Docker on your platform. Note that, as the instructions describe, once initially run, Docker can either be started by running the Boot2Docker applicaiton, or from the command line.

Mac users:
- Follow the instructions at https://docs.docker.com/installation/mac/ through **Basic Boot2Docker exercises**
Linux users:
- Follow the instructions at http://docs.docker.com/docker/installation/windows/ up to, but not including, "Further Details."
Linux users:
- Choose your platform under Install -> Linux, and follow the instructions.

If you followed the instructions, you will now have run your first Docker container, hello-world:

In [2]:
%%bash
docker run hello-world

Post http:///var/run/docker.sock/v1.19/containers/create: dial unix /var/run/docker.sock: no such file or directory. Are you trying to connect to a TLS-enabled daemon without TLS?


### How Docker runs on Mac and Windows
`DOCKER_HOST` is a VM layer between the host OS and the docker images: 
![Docker Host](http://docs.docker.com/installation/images/win_docker_host.svg)
![Docker Host](https://docs.docker.com/installation/images/mac_docker_host.svg)

In [3]:
boot2docker status

running


This tells us that `DOCKER_HOST` is running, though right now, there are no daemons running inside of it yet.

In [4]:
docker version

Client version: 1.7.1
Client API version: 1.19
Go version (client): go1.4.2
Git commit (client): 786b29d
OS/Arch (client): darwin/amd64
Server version: 1.7.1
Server API version: 1.19
Go version (server): go1.4.2
Git commit (server): 786b29d
OS/Arch (server): linux/amd64


In [23]:
docker run -d --name web nginx

5ace7dc12696625d8e2eed9635a71bd8fd325625f3fd86344ff07fba2bfc3288


In [24]:
docker ps

CONTAINER ID        IMAGE               COMMAND                CREATED             STATUS                  PORTS               NAMES
5ace7dc12696        nginx               "nginx -g 'daemon of   1 seconds ago       Up Less than a second   80/tcp, 443/tcp     web                 


In [25]:
docker port web

In [26]:
boot2docker ip

192.168.59.103


In [27]:
docker inspect web

[
{
    "Id": "5ace7dc12696625d8e2eed9635a71bd8fd325625f3fd86344ff07fba2bfc3288",
    "Created": "2015-07-22T18:04:08.241675766Z",
    "Path": "nginx",
    "Args": [
        "-g",
        "daemon off;"
    ],
    "State": {
        "Running": true,
        "Paused": false,
        "Restarting": false,
        "OOMKilled": false,
        "Dead": false,
        "Pid": 2622,
        "ExitCode": 0,
        "Error": "",
        "StartedAt": "2015-07-22T18:04:08.579087107Z",
        "FinishedAt": "0001-01-01T00:00:00Z"
    },
    "Image": "6886fb5a9b8d73b12d842bab8f9a6941c36094c2974abddb685d54c9d99e37da",
    "NetworkSettings": {
        "Bridge": "",
        "EndpointID": "7dd70c626487754480b125a8c0c3c7eaab83e3b804a02fb9a92d704d53ddae4b",
        "Gateway": "172.17.42.1",
        "GlobalIPv6Address": "",
        "GlobalIPv6PrefixLen": 0,
        "HairpinMode": false,
        "IPAddress": "172.17.0.7",
        "IPPrefixLen": 16,
        "IPv6Gateway": "",
        "LinkLocalIPv6Address": "",


In [20]:
docker inspect hello-world

[
{
    "Id": "91c95931e552b11604fea91c2f537284149ec32fff0f700a4769cfd31d7696ae",
    "Parent": "a8219747be10611d65b7c693f48e7222c0bf54b5df8467d3f99003611afa1fd8",
    "Comment": "",
    "Created": "2015-04-17T22:01:38.74474669Z",
    "Container": "724582a033a54c0864aab1e37d7bfaea9a3228bc63282df0dd07dd5768da40be",
    "ContainerConfig": {
        "Hostname": "d7c065bb52d8",
        "Domainname": "",
        "User": "",
        "AttachStdin": false,
        "AttachStdout": false,
        "AttachStderr": false,
        "PortSpecs": null,
        "ExposedPorts": null,
        "Tty": false,
        "OpenStdin": false,
        "StdinOnce": false,
        "Env": null,
        "Cmd": [
            "/bin/sh",
            "-c",
            "#(nop) CMD [\"/hello\"]"
        ],
        "Image": "a8219747be10611d65b7c693f48e7222c0bf54b5df8467d3f99003611afa1fd8",
        "Volumes": null,
        "VolumeDriver": "",
        "WorkingDir": "",
        "Entrypoint": null,
        "NetworkDisabled": fal

Notice that there are no `NetworkSettings` for `hello-world` because it is not running.

In [28]:
docker stop web

web


In [29]:
docker rm web

web


---

# Contents of a Dockerfile
Let's take a look at this Dockerfile

In [1]:
%%bash
cat Dockerfile

# We use the CentOS version 7 as a base for our own image that is described by this file.
FROM centos:7
MAINTAINER Galvanize

RUN echo "Expose SSH port"
EXPOSE 22 

RUN echo "Install openssh"
RUN yum -y install openssl openssh-server passwd shadow-utils sudo && yum clean all

RUN echo "Install basic packages"
RUN yum -y install vim nano wget 

RUN echo "Configure root Account"
RUN mkdir /var/run/sshd
RUN echo 'root:root' | chpasswd

RUN echo "SSH login fix for Docker"
RUN sed -i 's/PermitRootLogin without-password/PermitRootLogin yes/' /etc/ssh/sshd_config
RUN sed 's@session\s*required\s*pam_loginuid.so@session optional pam_loginuid.so@g' -i /etc/pam.d/sshd
ENV NOTVISIBLE "in users profile"
RUN echo "export VISIBLE=now" >> /etc/profile
RUN echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config

RUN echo "Setup gstudent user with sudo-rights (no password needed)"
RUN useradd -m -d /home/gstudent -s /bin/bash -c "gstudent user" -p $(openssl passwd -1 gstudent)  gstudent
RUN echo "gstud

Copy your public key into `tmp/authorized_keys`  
*e.g.*:
```bash
mkdir tmp
cp ~/.ssh/id_rsa.pub tmp/authorized_keys
```

In [32]:
docker build -t testimg .



Step 0 : FROM centos:7
7: Pulling from centos



centos:7: The image you are pulling has been verified. Important: image verification is a tech preview feature and should not be relied on to provide security.
Digest: sha256:c65da3ab236f6368a3e2722a6f576b80f6e1de25067612ca2b55506d9b554dba
Status: Downloaded newer image for centos:7
 ---> 7322fbe74aa5
Step 1 : MAINTAINER Galvanize
 ---> Running in c13ba6ea49a6
 ---> 1d6d36d746ee
Removing intermediate container c13ba6ea49a6
Step 2 : RUN echo "Expose SSH port"
 ---> Running in 45fc45d7d5b2
Expose SSH port
 ---> 6f3cf19dfd0d
Removing intermediate container 45fc45d7d5b2
Step 3 : EXPOSE 22
 ---> Running in 79c1a220d516
 ---> d7725800d30d
Removing intermediate container 79c1a220d516
Step 4 : RUN echo "Install openssh"
 ---> Running in 34a41ad1ab29
Install openssh
 ---> 805f42d5d334
Removing intermediate container 34a41ad1ab29
Step 5 : RUN yum -y install openssl openssh-server passwd shadow-utils sudo && yum clean all
 ---> Running in 7441007

In [33]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
testimg             latest              56786ac35538        16 minutes ago      195.1 MB
nginx               latest              6886fb5a9b8d        4 days ago          132.9 MB
centos              7                   7322fbe74aa5        4 weeks ago         172.2 MB
hello-world         latest              91c95931e552        3 months ago        910 B


In [35]:
docker run -d -p 22 --privileged -h test --name test testimg

dff3a4f87392642bf84822c5049ce6c4431d8c6844f0d05c669eb4bc6e1e1a1d


In [37]:
docker inspect test

[
{
    "Id": "dff3a4f87392642bf84822c5049ce6c4431d8c6844f0d05c669eb4bc6e1e1a1d",
    "Created": "2015-07-22T18:54:31.784128709Z",
    "Path": "/usr/sbin/sshd",
    "Args": [
        "-D"
    ],
    "State": {
        "Running": true,
        "Paused": false,
        "Restarting": false,
        "OOMKilled": false,
        "Dead": false,
        "Pid": 3458,
        "ExitCode": 0,
        "Error": "",
        "StartedAt": "2015-07-22T18:54:32.113814411Z",
        "FinishedAt": "0001-01-01T00:00:00Z"
    },
    "Image": "56786ac3553853c22d5124bafafade006f2d03d37f7bf55cb0a39b5b54bfd51d",
    "NetworkSettings": {
        "Bridge": "",
        "EndpointID": "d4b5cf7953149bc44c759f554a934412ebb437bcc9da3209df6e9fbb06f75a61",
        "Gateway": "172.17.42.1",
        "GlobalIPv6Address": "",
        "GlobalIPv6PrefixLen": 0,
        "HairpinMode": false,
        "IPAddress": "172.17.0.30",
        "IPPrefixLen": 16,
        "IPv6Gateway": "",
        "LinkLocalIPv6Address": "",
        "Link

Get IPAddress from `docker inspect` and put it in
```bash
sudo route -n add {$IPAddress}/16 `boot2docker ip`
```

for example, if your `"IPAddress": "172.17.0.30"`:
```bash
sudo route -n add 172.17.0.30/16 `boot2docker ip`
```

test:
```bash
ping -o 172.17.0.30 
```
finally:
```bash
ssh gstudent@172.17.0.30
```

# Ansible: Installation and basic commands
Ansible only needs to be installed on the machine from which it operates, and does not need to be installed on machines it executes tassk on. 
On either Mac or Linux, install Ansible using the appropriate package manager.

In [ ]:
# todo: use pip install for Ansible for cross-platform consistency
sudo apt-add-repository -y ppa:ansible/ansible
sudo apt-get update
sudo apt-get install -y ansible

Let's try the simplest possible Ansible command:

In [ ]:
ansible all -i 'localhost,' -c local -m ping

In [ ]:
localhost | success >> {
    "changed": false, 
    "ping": "pong"
}

Here's what this line did:
- 'all': tells ansible to run the task against all hosts in its inventory
- '-i localhost,': the -i flag provides ansible with either the path to an inventory file, OR a list of host names (NOTE: the comma tells Ansible we're providing a list instead of a filepath)
- '-c local': use local connection instead of SSH'ing into any remote hosts.
- '-m ping': use the 'ping' module.


## Ansible ad-hoc command exercises:
Use Ansible ad-hoc commands to:
- gather facts about a running Docker container
- transfer a file from your machine to the Docker container.
- install a package, and then install the same package 
  - Ansible is idempotent, meaning that the effect of a command is the same whether you run it once or more than once
- manage services on the container (start, stop, restart)




Here is an example of a simple Hosts file. Comments describe the contents.

```yaml
## group name, e.g. 'local', 'webservers', 'hadoop-cluster':
[local] 
## hostname, ip address, ssh into the machine, log in as user gstudent:
test ansible_ssh_host=172.17.0.132 ansible_ssh_user=gstudent 
```

Here is an example of an Ansible Playbook. This playbook:
- applies a set of tasks to all hosts defined in the hosts file with user gstudent which we set up in the Dockerfile
- sets variables for repeated use
- installs Java and makes it available throughout the system

```yaml
---
- hosts: all
  remote_user: gstudent
  vars:
    download_url: http://download.oracle.com/otn-pub/java/jdk/8u51-b16/jdk-8u51-linux-x64.tar.gz
    download_folder: /opt
    java_name: "{{download_folder}}/jdk1.8.0_05"
    java_archive: "{{download_folder}}/jdk-8u51-linux-x64.tar.gz"


  tasks:    
  - name: Download Java
    command: "sudo wget --no-cookies --no-check-certificate --header "Cookie: gpw_e24=http://www.oracle.com/; oraclelicense=accept-securebackup-cookie" "http://download.oracle.com/otn-pub/java/jdk/8u20-b26/jdk-8u20-linux-x64.tar.gz"

  - name: Unpack archive
    command: "tar -zxf {{java_archive}} -C {{download_folder}} creates={{java_name}}"

  - name: Fix ownership
     file: state=directory path={{java_name}} owner=root group=root recurse=yes

  - name: Make Java available for system
     command: 'alternatives --install "/usr/bin/java" "java" "{{java_name}}/bin/java" 2000'

  - name: Clean up
     file: state=absent path={{java_archive}}
```

Next step, we install Postgres as a database:

```yaml
# Create a new database with name "acme"
- postgresql_db: name=acme

# Create a new database with name "acme" and specific encoding and locale
# settings. If a template different from "template0" is specified, encoding
# and locale settings must match those of the template.
- postgresql_db: name=acme
                 encoding='UTF-8'
                 lc_collate='de_DE.UTF-8'
                 lc_ctype='de_DE.UTF-8'
                 template='template0'
```

Now, create a roles directory, and within that a 'common' and 'database'. 

## Ansible directory structure
This is what the directory structure for a web application might look like. Notice:
- separation of production and staging environments
- roles for common components, as well as webtier, monitoring and the app itself

```yaml
---
production                # inventory file for production servers
staging                   # inventory file for staging environment

group_vars/
   group1                 # here we assign variables to particular groups
   group2                 # ""
host_vars/
   hostname1              # if systems need specific variables, put them here
   hostname2              # ""

library/                  # if any custom modules, put them here (optional)
filter_plugins/           # if any custom filter plugins, put them here (optional)

site.yml                  # master playbook
webservers.yml            # playbook for webserver tier
dbservers.yml             # playbook for dbserver tier

roles/
    common/               # this hierarchy represents a "role"
        tasks/            #
            main.yml      #  <-- tasks file can include smaller files if warranted
        handlers/         #
            main.yml      #  <-- handlers file
        templates/        #  <-- files for use with the template resource
            ntp.conf.j2   #  <------- templates end in .j2
        files/            #
            bar.txt       #  <-- files for use with the copy resource
            foo.sh        #  <-- script files for use with the script resource
        vars/             #
            main.yml      #  <-- variables associated with this role
        defaults/         #
            main.yml      #  <-- default lower priority variables for this role
        meta/             #
            main.yml      #  <-- role dependencies

    webtier/              # same kind of structure as "common" was above, done for the webtier role
    monitoring/           # ""
    fooapp/     
```

## EC2 provisioning example
This is simple example of a playbook that provisions a machine on Amazon EC2. 

```yaml
- ec2:
    key_name: mykey 
    instance_type: t2.micro
    image: ami-123456
    wait: yes
    group: webserver
    count: 3
    vpc_subnet_id: subnet-29e63245
    assign_public_ip: yes
```

## EC2 deployment exercises:
1. Go to AWS console, start 4 nodes with Red Hat image
2. Create an Ansible hosts file with a group [aws] and add the public IP addresses of each of the machines
3. Create Ansible roles for Postgres master+slave, Java, Apache
4. Create a basic Ansible playbook that executes against this group, and adds appropriate roles for each
5. Install Ansible ec2 module 
6. Use Ansible's ec2 module to set up provisioning for each EC2 machine
  - add ec2 configuration parameters to playbook
7. Run the Playbook and go to the public page of the Apache host: it should show the welcome html page


(1) AWS Console:
    1. Go to console: https://aws.amazon.com/console/
    2. Register an AWS account and get a Free Tier
    3. Go to EC2 and click 'Instances'
    4. Start 4 machines with Red Hat image
    5. Copy their public IP addresses

(2) Create an Ansible hosts file with a group [aws] and add the public IP addresses of each of the machines

```yaml
[aws]
ec2-50-19-163-42.compute-1.amazonaws.com ansible_ssh_private_key_file=ansible_ec2.pem
```

    (3) Create Ansible roles for Postgres master+slave, Java, Apache

```yaml

# role: Postgres
  vars:
    dbname: gdb
    # dbuser: gdbuser
    dbuser: postgres
    # dbpassword: gdbpwd
    dbpassword: 

  tasks:
  - name: Install postgresql with dependencies
    yum: pkg={{item}} state=installed
    with_items:
        - postgresql
        - python-psycopg2


# role: Java
- name: Download JDK (CentOS)
  shell: "wget --quiet -O {{ jdk_download_folder }}/{{ jdk_archive }} {{ jdk_download_url }}"
  sudo: yes
  args:
    creates: '{{ jdk_download_folder }}/{{ jdk_archive }}'

- name: Install JDK (CentOS)
  shell: "yum -y --nogpgcheck localinstall {{jdk_download_folder}}/{{ jdk_archive }}" 
  args:
    creates: /usr/java
  sudo: yes

- name: symlink java (CentOS)
  file: src=/usr/java/default/bin/java dest=/usr/bin/java state=link
  sudo: yes

- name: symlink javac (CentOS)
  file: src=/usr/java/default/bin/javac dest=/usr/bin/javac state=link
  sudo: yes


# role: Apache
```

(4) Create an Ansible playbook that executes against this group, and adds appropriate roles for each

```yaml
# Variables listed here are applicable to all host groups
key_name: ec2-prod-key
aws_region: us-west-2
ami_id: ami-cc8de6fc
instance_type: t1.micro
    
    # Basic provisioning example
- name: Create AWS resources
  hosts: localhost
  connection: local
  gather_facts: False
  tasks:
  - name: Create security group
      module: ec2_group
      name: *my-security-group*
      description: "A Security group"
      region: "{{aws_region}}"
      rules:
        - proto: tcp
          type: ssh
          from_port: 22
          to_port: 22
          cidr_ip: 0.0.0.0/0
      rules_egress:
        - proto: all
          type: all
          cidr_ip: 0.0.0.0/0
    register: basic_firewall

  - name: create an EC2 instance
    local_action:
      module: ec2
      key_name: "{{key_name}}"
      region: "{{aws_region}}"
      group_id: "{{basic_firewall.group_id}}"
      instance_type: "{{instance_type}}"
      image: "{{ami_id}}"
      wait: yes
    register: basic_ec
```

7. Run the Playbook and go to the public page of the Apache host: it should show the welcome html page

```bash
$ ansible-playbook playbook-aws.yml -l aws-cluster
```